In [1]:
import json
import pathlib
from gpt3 import MewlCaptioner
from tqdm import tqdm

In [2]:
captioner = MewlCaptioner()

In [3]:
def test_task(captioner, dataset_path, task_name):
    dataset_path = pathlib.Path(dataset_path) / task_name
    all_episodes = dataset_path.glob('*/')
    all_episodes = sorted(all_episodes, key=lambda x: int(x.name))

    all_datasets = []

    caption_func = getattr(captioner, task_name)

    for i, epi in tqdm(enumerate(all_episodes), total=len(all_episodes)):
        prompt = ""
        contexts, query = caption_func(epi)
        for context in contexts:
            prompt += f"Context: {context[0]}\nName: {context[1]}\n\n"

        prompt += f"Context: {query[0]}\nName: "

        query_description, choices, answer = query

        label = choices.index(answer)

        all_datasets.append({
            "start_phrase": prompt,
            "choices": choices,
            "label": label,
            "task": task_name
        })

    return all_datasets

In [6]:
dataset_path = pathlib.Path("/home/guangyuan/MEWL")

In [7]:
from model.consts import task_names

for task in task_names:
    dataset = {}
    for split in ["train", "val", "test"]:
        dataset[split] = test_task(captioner, dataset_path / split, task)

    with open(f"./dataset/{task}.json", "w+") as f:
        json.dump(dataset, f, indent=4)

100%|██████████| 600/600 [00:00<00:00, 1699.29it/s]
